In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv(r"C:\Users\offic\Desktop\Code\AutoRecon\Data\testdata.csv")  # Change path as needed

# Display the first few rows (optional)
print(df.head())

# Count number of 'Withdrawn' and 'Captured' in 'Cashflow'
cashflow_counts = df['Cashflow'].value_counts()

# Show results
print("Transaction Type Counts:")
print(cashflow_counts)

# Or to extract specifically:
num_withdrawn = cashflow_counts.get('Withdrawn', 0)
num_deposits = cashflow_counts.get('Captured', 0)

print(f"\nNumber of Withdrawn transactions: {num_withdrawn}")
print(f"Number of Deposit transactions (Captured): {num_deposits}")


   ClientID    Client Name    PayerID            UserName  Paymentref  \
0      9884     iGi-gmycdy  670066766        nv1a78825311  7075137677   
1      9884     iGi-gmycdy  706587766  nv1anvDanie4691356  7074877070   
2      9884     iGi-gmycdy  706589608  nv1anvAshle0409812  7074854338   
3      9999   iGi-iichimqn  609876960      mi1aJafara9707  7074038537   
4      9880  iGi-iIiiiiiii  678876494  mf1aSamuel07382mf2  7074547870   

                              Uniqueref  Txn Date   Cashflow  PaymentAmount  \
0  A48E6u66-aE91-4aa1-82AE-352345818B3A  5/6/2025  Withdrawn         9800.0   
1  86a9A1uE-6AB5-4518-9AaB-u9u201B2u3D4  5/6/2025   Captured           50.0   
2  B06aa113-B554-44aA-916B-E0B1uE63818E  5/6/2025   Captured           25.0   
3  aEaDD0Eu-5082-4a26-9EB8-0384a894686a  5/6/2025  Withdrawn          300.0   
4  8280aEB8-a442-4392-BB31-a5B0B832uaD4  5/6/2025   Captured           50.0   

  Currency PayDirection  ProviderID PaymentProviderDesc  ProviderID.1  
0      USD   W

In [5]:
import pandas as pd


# Ensure no missing values interfere
df = df.dropna(subset=['Cashflow', 'PaymentAmount'])

# Sum amounts by Cashflow type
total_by_cashflow = df.groupby('Cashflow')['PaymentAmount'].sum()

# Extract specific totals
total_withdrawn = total_by_cashflow.get('Withdrawn', 0)
total_deposit = total_by_cashflow.get('Captured', 0)

# Display results
print(f"Total Withdrawn Amount: ${total_withdrawn:,.2f}")
print(f"Total Deposit (Captured) Amount: ${total_deposit:,.2f}")


Total Withdrawn Amount: $8,196,620.72
Total Deposit (Captured) Amount: $396,283.00


In [6]:
import pandas as pd

# Step 1: Load the dataset
df = pd.read_csv(r"C:\Users\offic\Desktop\Code\AutoRecon\Data\testdata.csv")  # Replace with your actual path if needed

# Step 2: Filter for users who performed both Withdrawn and Captured
withdrawn_df = df[df['Cashflow'] == 'Withdrawn']
captured_df = df[df['Cashflow'] == 'Captured']

withdraw_users = set(withdrawn_df['UserName'])
captured_users = set(captured_df['UserName'])

both_users = withdraw_users.intersection(captured_users)

# Step 3: Summarize total deposits and withdrawals per user
filtered_df = df[df['UserName'].isin(both_users)]

user_summary = (
    filtered_df.groupby(['UserName', 'Cashflow'])['PaymentAmount']
    .sum()
    .unstack(fill_value=0)
    .reset_index()
)

# Rename columns
user_summary = user_summary.rename(columns={
    'Captured': 'Total_Deposited',
    'Withdrawn': 'Total_Withdrawn'
})

# Step 4: Reconciliation logic
user_summary['Reconciled'] = user_summary['Total_Deposited'].round(2) == user_summary['Total_Withdrawn'].round(2)
user_summary['Difference'] = (user_summary['Total_Deposited'] - user_summary['Total_Withdrawn']).round(2)

# Step 5: Print unreconciled results
unreconciled_users = user_summary[~user_summary['Reconciled']]

print("Unreconciled Users:")
print(unreconciled_users[['UserName', 'Total_Deposited', 'Total_Withdrawn', 'Difference']])


Unreconciled Users:
Cashflow            UserName  Total_Deposited  Total_Withdrawn  Difference
0          mi1aRonald13752mi            300.0          4500.00    -4200.00
1               nv1a68636114            400.0           707.69     -307.69
2               nv1a92828302            500.0          1010.00     -510.00
3         nv1aWendell27681mf             50.0           100.00      -50.00
4            nv1aZachary7660            300.0           500.00     -200.00
5         nv1alaTrent5630276             60.0            40.00       20.00
6          nv1anvAdam9095881            800.0          1000.00     -200.00
7         nv1anvAllis7837361             60.0           216.00     -156.00
9         nv1anvAndre6838014            250.0           454.55     -204.55
10        nv1anvAntho1370910            100.0           400.00     -300.00
11        nv1anvAvary9383345             20.0            88.00      -68.00
12        nv1anvConno5226556            250.0           509.00     -259.00
13   

In [9]:
import pandas as pd
import random

# Load the uploaded CSV file
df = pd.read_csv(r"C:\Users\offic\Desktop\Code\AutoRecon\Data\testdata.csv")  # Replace with your actual path if needed


# Shuffle and sample 80% of the data to create a duplicate with 20% missing
df_sampled = df.sample(frac=0.8, random_state=42).reset_index(drop=True)

# Save the sampled DataFrame as the mismatched duplicate
duplicate_path = "testdata_duplicate_with_20_percent_missing.csv"
df_sampled.to_csv(duplicate_path, index=False)

duplicate_path


'testdata_duplicate_with_20_percent_missing.csv'

In [10]:
import pandas as pd

# Load your combined dataset
df = pd.read_csv(r"C:\Users\offic\Desktop\Code\AutoRecon\Data\testdata.csv")
df.columns = df.columns.str.strip()

# Assume internal reference is 'Paymentref' and external is 'ProviderID.1'
# Clean up types and remove nulls
internal = df[['ClientID', 'Paymentref', 'Txn Date', 'PaymentAmount', 'Currency', 'PayDirection']].dropna(subset=['Paymentref'])
external = df[['ClientID', 'ProviderID.1', 'Txn Date', 'PaymentAmount', 'Currency', 'PayDirection']].dropna(subset=['ProviderID.1'])

# Rename for clarity before merging
internal = internal.rename(columns={'Paymentref': 'Ref', 'Txn Date': 'Date', 'PaymentAmount': 'Amount'})
external = external.rename(columns={'ProviderID.1': 'Ref', 'Txn Date': 'Date', 'PaymentAmount': 'Amount'})

# Merge on 'Ref'
merged = pd.merge(internal, external, on='Ref', suffixes=('_int', '_ext'), how='outer', indicator=True)

# Define status
def get_status(row):
    if row['_merge'] == 'both':
        if row['Amount_int'] != row['Amount_ext'] or row['PayDirection_int'] != row['PayDirection_ext']:
            return 'Mismatch'
        return 'Match'
    elif row['_merge'] == 'left_only':
        return 'Missing in External'
    else:
        return 'Missing in Internal'

merged['Status'] = merged.apply(get_status, axis=1)

# Output sample
print(merged[['Ref', 'Amount_int', 'Amount_ext', 'PayDirection_int', 'PayDirection_ext', 'Status']])

            Ref  Amount_int  Amount_ext PayDirection_int PayDirection_ext  \
0      77745777         NaN   500000.00              NaN       Withdrawal   
1      77747674         NaN      100.00              NaN          Deposit   
2     777447658         NaN     3700.00              NaN       Withdrawal   
3     777447778         NaN      200.00              NaN          Deposit   
4     777457674         NaN       25.00              NaN          Deposit   
..          ...         ...         ...              ...              ...   
923  7777966847         NaN      100.00              NaN       Withdrawal   
924  7778497787         NaN      250.00              NaN       Withdrawal   
925  7778658857         NaN     2800.00              NaN       Withdrawal   
926  7778777658         NaN      115.00              NaN       Withdrawal   
927  7778857857         NaN      228.63              NaN       Withdrawal   

                  Status  
0    Missing in Internal  
1    Missing in Inter

In [4]:
import pandas as pd

# Load both CSV files

original_df = pd.read_csv(r"C:\Users\offic\Desktop\Code\AutoRecon\Data\NEWNEW.csv")
duplicate_df = pd.read_csv(r"C:\Users\offic\Desktop\Code\AutoRecon\Augementation\testdata_duplicate_with_20_percent_missing.csv")

# Reconciliation logic: find rows in original that are not in duplicate
# This assumes a full row match for uniqueness. You can change to use 'Transaction ID' or similar if available.
missing_transactions = pd.concat([original_df, duplicate_df, duplicate_df]).drop_duplicates(keep=False)

# Show the missing transactions
print("Missing Transactions:")
print(missing_transactions)

# Save to a new CSV file
missing_transactions.to_csv('missing_transactions.csv', index=False)
print("Reconciliation complete. Missing transactions saved to 'missing_transactions.csv'.")


Missing Transactions:
     ClientID      Client Name    PayerID            UserName  Paymentref  \
1        9884       iGi-gmycdy  706587766  nv1anvDanie4691356  7074877070   
12       9846  iGi-iueoto oico  648805659  pu1apuJorge5958561  7074973748   
19       9884       iGi-gmycdy  706598548   nv1anvAlex8400851  7075073918   
20       9884       iGi-gmycdy  706597040  nv1anvJeffr9812634  7074993507   
33       9999     iGi-iichimqn  667899865    mi1aLarry74642mi  7074789058   
..        ...              ...        ...                 ...         ...   
447      9849     iGi-iyoylynd  680867785    md1aDaniel182077  7074563441   
452      9884       iGi-gmycdy  706588865  nv1anvDenis3543243  7075140490   
455      9884       iGi-gmycdy  706598767  nv1ancAlsto3251726  7074937075   
459      9884       iGi-gmycdy  676968998  nv1anvMicha7324713  7074934370   
462      9884       iGi-gmycdy  697807656  nv1anvStefa5325536  7074708798   

                                Uniqueref  Txn Date  